In [ ]:
import sys
import os
import math
from time import sleep
from pythreejs import *

sys.path.append('../build/mechsystem') 

from mass_spring import *

In [ ]:
def add_beam(mss, node1, node2, length, stiffness, use_constraint=True):
    if use_constraint:
        dc = DistanceConstraint(node1, node2, length) #we use Lagrange constraints for make it more rigid and good
        mss.addDistanceConstraint(dc)
    else:
        mss.add(Spring(length, stiffness, (node1, node2))) #more elastic for cables

def build_crane():
    mss = MassSpringSystem3d()
    mss.gravity = (0.0, 0.0, -9.81) 

    # Parameters
    mass = 1.0    
    stiffness_beam = 5000.0 # hight for the robustness 
    stiffness_cable = 3000.0  # cabel must be elastic
    L = 1.0
    floors = 10 #how tall we want the crane to be

    # 1. Create the "tower" of the crane
    nodes = []

    for i in range(floors + 1):
        z = i * L
        floor_nodes = []
        
        coords = [    # We put one corner in the origin and define the others in the x>0, y>0 plane
            (0, 0, z),
            (L, 0, z), 
            (L, L, z), 
            (0, L, z)   
        ]

        for pos in coords:
            if i == 0:
                c = mss.add(Fix(pos)) #the floor 0 does not move
            else:
                c = mss.add(Mass(mass, pos))
            floor_nodes.append(c) # we save the data corners
        nodes.append(floor_nodes)

    # We now connect every floor we have created
    for i in range(floors):
        current_floor = nodes[i]
        next_floor = nodes[i+1]

        for k in range(4):
            c_curr = current_floor[k]
            c_next = next_floor[k]
            c_curr_neighbor = current_floor[(k + 1) % 4] 
            c_next_neighbor = next_floor[(k + 1) % 4] #index for the next corner in the current flor
            # we have add %4 because the loop sums but for example the alst one is 3-0 not 3-4, so we want
            # the values in module 4

            add_beam(mss, c_curr, c_next, L, 0, use_constraint=True) # vertical union (like 0 with 4)

            add_beam(mss, c_next, c_next_neighbor, L, 0, use_constraint=True)  #horizontal union (like 0-1)

            diag_len = math.sqrt(2) * L  # we creatt the X in each face
            add_beam(mss, c_curr, c_next_neighbor, diag_len, stiffness_beam, use_constraint=False)
            add_beam(mss, c_curr_neighbor, c_next, diag_len, stiffness_beam, use_constraint=False)

    # 2. We constract the "arm" of the crane
    arm_length = 6  
    
    prev_face = [ #we "glue" the arm in the right face of last cube of the tower (floor m and floor m-1)
        nodes[floors-1][1],
        nodes[floors-1][2],
        nodes[floors][2],  
        nodes[floors][1] 
    ]
    
    for j in range(1, arm_length + 1):
        current_x = (L) + (j * L) 
        z_bot = (floors - 1) * L
        z_top = floors * L
        
        arm_coords = [
            (current_x, 0, z_bot),
            (current_x, L, z_bot),
            (current_x, L, z_top),
            (current_x, 0, z_top) 
        ]
        
        current_face = []
        for pos in arm_coords:
            c = mss.add(Mass(mass, pos)) 
            current_face.append(c)
            
        for k in range(4):
            c_old = prev_face[k]
            c_new = current_face[k]
            c_new_neighbor = current_face[(k+1)%4]
            c_old_neighbor = prev_face[(k+1)%4]  
            
            add_beam(mss, c_old, c_new, L, 0, True)# in the x direction 
            
            add_beam(mss, c_new, c_new_neighbor, L, 0, True) # the others (z and y)
            
            diag = math.sqrt(2) * L #fro the X structure
            add_beam(mss, c_old, c_new_neighbor, diag, stiffness_beam, False)
            add_beam(mss, c_old_neighbor, c_new, diag, stiffness_beam, False)
            
        prev_face = current_face

    # 3. Vibration
    # We put it at the end of the arm(X=current_x), but in the middle (Y=L/2) and colgando un poco (Z-1)
    load_x = L + arm_length * L
    load_pos = (load_x, L/2, (floors-1)*L - 2.0) 
    
    load_mass = mss.add(Mass(5.0, load_pos))
    
    tip_node_1 = prev_face[0]
    tip_node_2 = prev_face[1] 
    
    dist_h = math.sqrt( (L/2)**2 + 2.0**2 )
    
    mss.add(Spring(dist_h, stiffness_cable, (tip_node_1, load_mass))) #more elastic so they rebote more
    mss.add(Spring(dist_h, stiffness_cable, (tip_node_2, load_mass)))

    return mss

mss = build_crane()

In [ ]:
# meshes
masses_mesh = []
for m in mss.masses:
    masses_mesh.append(
        Mesh(SphereBufferGeometry(0.2, 8, 8),
             MeshStandardMaterial(color='red'),
             position=tuple(m.pos)))
    
fixes_mesh = []
for f in mss.fixes:
    fixes_mesh.append(
        Mesh(SphereBufferGeometry(0.3, 8, 8),
             MeshStandardMaterial(color='gray'),
             position=tuple(f.pos)))

# lines 
const_pos = []
for c in mss.constraints:
    p1 = mss.fixes[c.c1.nr].pos if c.c1.type == 1 else mss.masses[c.c1.nr].pos
    p2 = mss.fixes[c.c2.nr].pos if c.c2.type == 1 else mss.masses[c.c2.nr].pos
    const_pos.append([tuple(p1), tuple(p2)])

lines_constraints = LineSegments2(
    LineSegmentsGeometry(positions=const_pos),
    LineMaterial(linewidth=3, color='orange')
)

spring_pos = []
for s in mss.springs:
    p1 = mss.fixes[s.connectors[0].nr].pos if s.connectors[0].type == 1 else mss.masses[s.connectors[0].nr].pos
    p2 = mss.fixes[s.connectors[1].nr].pos if s.connectors[1].type == 1 else mss.masses[s.connectors[1].nr].pos
    spring_pos.append([tuple(p1), tuple(p2)])

lines_springs = LineSegments2(
    LineSegmentsGeometry(positions=spring_pos),
    LineMaterial(linewidth=1, color='cyan')
)



In [ ]:
# 3D photo
view_width = 800
view_height = 500
camera = PerspectiveCamera(position=[20, 10, 20], lookAt=[5, 5, 0], aspect=view_width/view_height)
key_light = DirectionalLight(position=[0, 20, 10])
ambient = AmbientLight(intensity=0.6)

scene = Scene(children=[*masses_mesh, *fixes_mesh, lines_constraints, lines_springs, camera, key_light, ambient, AxesHelper(2)])
controller = OrbitControls(controlling=camera)
renderer = Renderer(camera=camera, scene=scene, controls=[controller], width=view_width, height=view_height)

display(renderer)

In [ ]:
steps_per_frame = 2  # to make it easir for the computer to represent graphically, 1 of each 2 steps

for i in range(1000):
    mss.simulate(0.01, 2)

    if i % steps_per_frame == 0:
    
        for m, mesh in zip(mss.masses, masses_mesh):
            mesh.position = tuple(m.pos)
        
        new_c_pos = []
        for c in mss.constraints:
            p1 = mss.fixes[c.c1.nr].pos if c.c1.type == 1 else mss.masses[c.c1.nr].pos
            p2 = mss.fixes[c.c2.nr].pos if c.c2.type == 1 else mss.masses[c.c2.nr].pos
            new_c_pos.append([tuple(p1), tuple(p2)])
        lines_constraints.geometry = LineSegmentsGeometry(positions=new_c_pos)

        new_s_pos = []
        for s in mss.springs:
            p1 = mss.fixes[s.connectors[0].nr].pos if s.connectors[0].type == 1 else mss.masses[s.connectors[0].nr].pos
            p2 = mss.fixes[s.connectors[1].nr].pos if s.connectors[1].type == 1 else mss.masses[s.connectors[1].nr].pos
            new_s_pos.append([tuple(p1), tuple(p2)])
        lines_springs.geometry = LineSegmentsGeometry(positions=new_s_pos)
    
        sleep(0.01)